# 3D Visualization

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv

import torch.optim as optim
from rockgan.architecture import *
from rockgan.utils import *

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

DEVICE = torch.device('cuda')

## Training Data

In [3]:
DATASET = torch.from_numpy(np.load('../data/Berea_subpatches.npy')[:100])
print(DATASET.shape)

torch.Size([100, 128, 128, 128])


In [4]:
columns = 3
rows = 3
t = np.arange(1,rows*columns+1).reshape(rows,columns)
p = pv.Plotter(shape=(rows, columns))
for i in range(rows):
    for j in range(columns):
        grid = pv.UniformGrid(dims=(129,129,129))
        grid.cell_data["values"] = DATASET[t[i,j],:,:].flatten() 
        threshed = grid.threshold(value=0, invert=True)
        outline = grid.outline()
        p.subplot(i,j)
        p.add_mesh(outline, color="k")
        p.add_mesh(threshed, color='lightskyblue',interpolate_before_map=True, show_edges=0, smooth_shading=True, metallic=1)
        p.add_text('Training sample: N= {}'.format(t[i,j]), font_size=12, font='times', color='black')
#         p.camera_position = [-5, 5, 5]
        p.window_size = [1000,1000]
        #p.camera.zoom(1.1)
# p.save_graphic('../figures/training_samples.eps',raster=False )
# p.save_graphic('../figures/training_samples.eps')  
# p.screenshot('../figures/training_3D_samples_structure.png', transparent_background=True)
p.show()

2022-10-02 22:49:48.807 (   3.137s) [        C5871740]       vtkThreshold.cxx:72    WARN| vtkThreshold::ThresholdByLower was deprecated for VTK 9.1 and will be removed in a future version.
2022-10-02 22:49:49.352 (   3.681s) [        C5871740]       vtkThreshold.cxx:72    WARN| vtkThreshold::ThresholdByLower was deprecated for VTK 9.1 and will be removed in a future version.
2022-10-02 22:49:49.905 (   4.235s) [        C5871740]       vtkThreshold.cxx:72    WARN| vtkThreshold::ThresholdByLower was deprecated for VTK 9.1 and will be removed in a future version.
2022-10-02 22:49:50.446 (   4.776s) [        C5871740]       vtkThreshold.cxx:72    WARN| vtkThreshold::ThresholdByLower was deprecated for VTK 9.1 and will be removed in a future version.
2022-10-02 22:49:50.994 (   5.323s) [        C5871740]       vtkThreshold.cxx:72    WARN| vtkThreshold::ThresholdByLower was deprecated for VTK 9.1 and will be removed in a future version.
2022-10-02 22:49:51.539 (   5.869s) [        C5871740] 

ViewInteractiveWidget(height=1000, layout=Layout(height='auto', width='100%'), width=1000)

In [5]:
columns = 3
rows = 3
t = np.arange(1,rows*columns+1).reshape(rows,columns)
p = pv.Plotter(shape=(rows, columns))

for i in range(rows):
    for j in range(columns):
        grid = pv.UniformGrid(dims=(129,129,129))
        grid.cell_data["values"] = DATASET[t[i,j],:,:].flatten() 
        clipped = grid.clip_box(factor=0.5)
        p.subplot(i,j)
        p.add_mesh(clipped, cmap='gray', show_scalar_bar=False)
        p.add_text('Training sample: N= {}'.format(t[i,j]), font_size=12, font='times', color='black')
#         p.camera_position = [-5, 5, 5]
        p.window_size = [1000,1000]
        #p.camera.zoom(1.1)
# p.save_graphic('../figures/training_samples.eps',raster=False )
# p.save_graphic('../figures/training_samples.eps')  
# p.screenshot('../figures/training_3D_samples_cube.png', transparent_background=True)
p.show()

ViewInteractiveWidget(height=1000, layout=Layout(height='auto', width='100%'), width=1000)

## RockGAN results

In [6]:
# Training hyperparameters
LEARNING_RATE = 1e-3
BATCH_SIZE = 32
Z_DIM = 16
# fixed noise for display
fixed_noise = torch.randn(BATCH_SIZE,1, Z_DIM, Z_DIM, Z_DIM).to(DEVICE)
# generator
gen = Generator(in_channel=1, out_channel=1).to(DEVICE)
# optimizer
opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.0, 0.9))

In [7]:
#Loading generator per epochs
load_checkpoint("../checkpoints/generator/generator_no_poro_99_V2.pt", \
                model=gen, optimizer=opt_gen, lr=1e-3)
RockGAN_images = np.round(gen(fixed_noise).detach().cpu().numpy().reshape(BATCH_SIZE,128,128,128))

=> Loading checkpoint


In [8]:
columns = 3
rows = 3
t = np.arange(1,rows*columns+1).reshape(rows,columns)
p = pv.Plotter(shape=(rows, columns))
for i in range(rows):
    for j in range(columns):
        grid = pv.UniformGrid(dims=(129,129,129))
        grid.cell_data["values"] = RockGAN_images[t[i,j],:,:].flatten() 
        threshed = grid.threshold(value=0, invert=True)
        outline = grid.outline()
        p.subplot(i,j)
        p.add_mesh(outline, color="k")
        p.add_mesh(threshed, color='lightskyblue',interpolate_before_map=True, show_edges=0, smooth_shading=True, metallic=1)
        p.add_text('RockGAN sample: N= {}'.format(t[i,j]), font_size=12, font='times', color='black')
#         p.camera_position = [-5, 5, 5]
        p.window_size = [1000,1000]
        #p.camera.zoom(1.1)
# p.save_graphic('../figures/RockGAN_3D_samples.eps',raster=False )
# p.save_graphic('../figures/RockGAN_3D_samples.eps')  
# p.screenshot('../figures/RockGAN_3D_samples_structure.png', transparent_background=True)
p.show()

2022-10-02 22:50:58.629 (  72.959s) [        C5871740]       vtkThreshold.cxx:72    WARN| vtkThreshold::ThresholdByLower was deprecated for VTK 9.1 and will be removed in a future version.
2022-10-02 22:50:59.186 (  73.515s) [        C5871740]       vtkThreshold.cxx:72    WARN| vtkThreshold::ThresholdByLower was deprecated for VTK 9.1 and will be removed in a future version.
2022-10-02 22:50:59.701 (  74.031s) [        C5871740]       vtkThreshold.cxx:72    WARN| vtkThreshold::ThresholdByLower was deprecated for VTK 9.1 and will be removed in a future version.
2022-10-02 22:51:00.222 (  74.552s) [        C5871740]       vtkThreshold.cxx:72    WARN| vtkThreshold::ThresholdByLower was deprecated for VTK 9.1 and will be removed in a future version.
2022-10-02 22:51:00.755 (  75.085s) [        C5871740]       vtkThreshold.cxx:72    WARN| vtkThreshold::ThresholdByLower was deprecated for VTK 9.1 and will be removed in a future version.
2022-10-02 22:51:01.279 (  75.608s) [        C5871740] 

ViewInteractiveWidget(height=1000, layout=Layout(height='auto', width='100%'), width=1000)

In [9]:
columns = 3
rows = 3
t = np.arange(1,rows*columns+1).reshape(rows,columns)
p = pv.Plotter(shape=(rows, columns))

for i in range(rows):
    for j in range(columns):
        grid = pv.UniformGrid(dims=(129,129,129))
        grid.cell_data["values"] = RockGAN_images[t[i,j],:,:].flatten() 
        clipped = grid.clip_box(factor=0.5)
        p.subplot(i,j)
        p.add_mesh(clipped, cmap='gray', show_scalar_bar=False)
        p.add_text('RockGAN sample: N= {}'.format(t[i,j]), font_size=12, font='times', color='black')
#         p.camera_position = [-5, 5, 5]
        p.window_size = [1000,1000]
        #p.camera.zoom(1.1)
# p.save_graphic('../figures/training_samples.eps',raster=False )
# p.save_graphic('../figures/training_samples.eps')  
# p.screenshot('../figures/RockGAN_3D_samples_cube.png', transparent_background=True)
p.show()

ViewInteractiveWidget(height=1000, layout=Layout(height='auto', width='100%'), width=1000)

## CRockGAN results

In [10]:
#Loading generator per epochs
load_checkpoint("../checkpoints/generator/generator_poro_99_v4.pt", \
                model=gen, optimizer=opt_gen, lr=1e-3)
CRockGAN_images = np.round(gen(fixed_noise).detach().cpu().numpy().reshape(BATCH_SIZE,128,128,128))

=> Loading checkpoint


In [11]:
columns = 3
rows = 3
t = np.arange(1,rows*columns+1).reshape(rows,columns)
p = pv.Plotter(shape=(rows, columns))
for i in range(rows):
    for j in range(columns):
        grid = pv.UniformGrid(dims=(129,129,129))
        grid.cell_data["values"] = CRockGAN_images[t[i,j],:,:].flatten() 
        threshed = grid.threshold(value=0, invert=True)
        outline = grid.outline()
        p.subplot(i,j)
        p.add_mesh(outline, color="k")
        p.add_mesh(threshed, color='lightskyblue',interpolate_before_map=True, show_edges=0, smooth_shading=True, metallic=1)
        p.add_text('CRockGAN sample: N= {}'.format(t[i,j]), font_size=12, font='times', color='black')
#         p.camera_position = [-5, 5, 5]
        p.window_size = [1000,1000]
        #p.camera.zoom(1.1)
# p.save_graphic('../figures/RockGAN_3D_samples.eps',raster=False )
# p.save_graphic('../figures/RockGAN_3D_samples.eps')  
# p.screenshot('../figures/CRockGAN_3D_samples_structure.png', transparent_background=True)
p.show()

2022-10-02 22:52:04.456 ( 138.785s) [        C5871740]       vtkThreshold.cxx:72    WARN| vtkThreshold::ThresholdByLower was deprecated for VTK 9.1 and will be removed in a future version.
2022-10-02 22:52:05.008 ( 139.337s) [        C5871740]       vtkThreshold.cxx:72    WARN| vtkThreshold::ThresholdByLower was deprecated for VTK 9.1 and will be removed in a future version.
2022-10-02 22:52:05.546 ( 139.875s) [        C5871740]       vtkThreshold.cxx:72    WARN| vtkThreshold::ThresholdByLower was deprecated for VTK 9.1 and will be removed in a future version.
2022-10-02 22:52:06.080 ( 140.410s) [        C5871740]       vtkThreshold.cxx:72    WARN| vtkThreshold::ThresholdByLower was deprecated for VTK 9.1 and will be removed in a future version.
2022-10-02 22:52:06.620 ( 140.950s) [        C5871740]       vtkThreshold.cxx:72    WARN| vtkThreshold::ThresholdByLower was deprecated for VTK 9.1 and will be removed in a future version.
2022-10-02 22:52:07.159 ( 141.488s) [        C5871740] 

ViewInteractiveWidget(height=1000, layout=Layout(height='auto', width='100%'), width=1000)

In [12]:
columns = 3
rows = 3
t = np.arange(1,rows*columns+1).reshape(rows,columns)
p = pv.Plotter(shape=(rows, columns))

for i in range(rows):
    for j in range(columns):
        grid = pv.UniformGrid(dims=(129,129,129))
        grid.cell_data["values"] = CRockGAN_images[t[i,j],:,:].flatten() 
        clipped = grid.clip_box(factor=0.5)
        p.subplot(i,j)
        p.add_mesh(clipped, cmap='gray', show_scalar_bar=False)
        p.add_text('CRockGAN sample: N= {}'.format(t[i,j]), font_size=12, font='times', color='black')
#         p.camera_position = [-5, 5, 5]
        p.window_size = [1000,1000]
        #p.camera.zoom(1.1)
# p.save_graphic('../figures/training_samples.eps',raster=False )
# p.save_graphic('../figures/training_samples.eps')  
# p.screenshot('../figures/CRockGAN_3D_samples_cube.png', transparent_background=True)
p.show()

ViewInteractiveWidget(height=1000, layout=Layout(height='auto', width='100%'), width=1000)